# Load data

In [1]:
import pandas as pd 
import numpy as np
import glob
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


MC_DATA_FOLDER = '../results/MC_data'
TOTAL_NUMBER_OF_ANNIHILATIONS = 10_000_000
TOTAL_NUMBER_OF_PHOTONS = TOTAL_NUMBER_OF_ANNIHILATIONS * 2
Rs = [5.0, 10.0,15.0]
scatter_tbls  = {}
lor_tbls = {}

for R in Rs:
    try:
        lor_files = sorted(glob.glob(f"{MC_DATA_FOLDER}/lors_*_R{int(R)}.feather"))
        phot_files = sorted(glob.glob(f"{MC_DATA_FOLDER}/photons_*_R{int(R)}.feather"))

        lors_df = pd.concat((pd.read_feather(f) for f in lor_files), ignore_index=True)
        photons_df = pd.concat((pd.read_feather(f) for f in phot_files), ignore_index=True)
        print(f"R={R}")
        print('Number of LORs:', len(lors_df),"Number of photons", len(photons_df))
        scatter_tbls[R] = photons_df
        lor_tbls[R] = lors_df
    except Exception as e:
        print(e)


R=5.0
Number of LORs: 9931300 Number of photons 20000000
R=10.0
Number of LORs: 9389901 Number of photons 20000000
R=15.0
Number of LORs: 8324892 Number of photons 20000000


# Table of simulation results

### LaTex-friendly printout

### 

In [2]:
import numpy as np

def pct(n, denom):
    return 100.0 * n / denom if denom else float("nan")

def latex_int(n: int) -> str:
    return f"{int(n):,}".replace(",", r"\,")

def latex_pct(x: float) -> str:
    return f"{x:.1f}" + r"\%"

# Category order (stable output)
cat_names = ["Not scattered", "Scattered", "Absorbed", "Exited", "Emitted"]

# Collect stats per R
stats_by_R = {}

for R in Rs:
    lor = lor_tbls[R]
    N_emitted = TOTAL_NUMBER_OF_PHOTONS

    e1 = lor["exit_energy_1_keV"].to_numpy(float) # exited photon energy, is None if it were absorbed
    e2 = lor["exit_energy_2_keV"].to_numpy(float) # exited photon energy for photon 2, is None if it were absorbed
    N_exited = int(np.isfinite(e1).sum() + np.isfinite(e2).sum()) # count exited photons

    n1 = lor["n_scatter_p1"].to_numpy(int) # The number of times photon 1 scattered
    n2 = lor["n_scatter_p2"].to_numpy(int) # The number of times photon 2 scattered

    N_no_scatter = int((n1 == 0).sum() + (n2 == 0).sum()) # count photons that exited without scattering
    N_scattered  = int((n1 > 0).sum()  + (n2 > 0).sum()) # count photons that exited after scattering
    
    N_absorbed = int(N_emitted - N_exited) # Absorbed photons never show up as exited

    # --- sanity checks ---
    if N_no_scatter + N_scattered != N_exited:
        print(f"R={R}: no_scatter+scattered != exited "
              f"({N_no_scatter}+{N_scattered} != {N_exited})")
    if N_absorbed < 0:
        print(f"[WARN] R={R}: absorbed < 0 (check N_emitted and N_exited)")

    stats_by_R[R] = dict(
        emitted=N_emitted,
        exited=N_exited,
        absorbed=N_absorbed,
        no_scatter=N_no_scatter,
        scattered=N_scattered,
    )


# Create a LeTex printout that can be copy-pasted into a table
def cell(n, denom):
    return f"{latex_int(n)} ({latex_pct(pct(n, denom))})"

header = "Category & " + " & ".join([rf"$R={R:.0f}\,\mathrm{{cm}}$" for R in Rs]) + r" \\"
print(header)
print(r"\midrule")

for name in cat_names:
    cells = []
    for R in Rs:
        s = stats_by_R[R]
        denom = s["emitted"]

        if name == "Not scattered":
            cells.append(cell(s["no_scatter"], denom))
        elif name == "Scattered":
            cells.append(cell(s["scattered"], denom))
        elif name == "Absorbed":
            cells.append(cell(s["absorbed"], denom))
        elif name == "Exited":
            cells.append(cell(s["exited"], denom))
        elif name == "Emitted":
            cells.append(f"{latex_int(s['emitted'])} (100.0\\%)")

    print(f"{name} & " + " & ".join(cells) + r" \\")




Category & $R=5\,\mathrm{cm}$ & $R=10\,\mathrm{cm}$ & $R=15\,\mathrm{cm}$ \\
\midrule
Not scattered & 10\,460\,209 (52.3\%) & 5\,945\,323 (29.7\%) & 3\,614\,886 (18.1\%) \\
Scattered & 9\,402\,391 (47.0\%) & 12\,834\,479 (64.2\%) & 13\,034\,898 (65.2\%) \\
Absorbed & 137\,400 (0.7\%) & 1\,220\,198 (6.1\%) & 3\,350\,216 (16.8\%) \\
Exited & 19\,862\,600 (99.3\%) & 18\,779\,802 (93.9\%) & 16\,649\,784 (83.2\%) \\
Emitted & 20\,000\,000 (100.0\%) & 20\,000\,000 (100.0\%) & 20\,000\,000 (100.0\%) \\


### Displayed

In [3]:
import numpy as np
import pandas as pd
from IPython.display import display

def pct(n, denom):
    return 100.0 * n / denom if denom else float("nan")

def fmt_int(n: int) -> str:
    return f"{int(n):,}"  # Python-style thousands separators

def fmt_pct(x: float) -> str:
    return f"{x:.1f}%"

cat_names = ["Not scattered", "Scattered", "Absorbed", "Exited", "Emitted"]

stats_by_R = {}

for R in Rs:
    lor = lor_tbls[R]
    N_emitted = TOTAL_NUMBER_OF_PHOTONS
    e1 = lor["exit_energy_1_keV"].to_numpy(float)
    e2 = lor["exit_energy_2_keV"].to_numpy(float)
    N_exited = int(np.isfinite(e1).sum() + np.isfinite(e2).sum())
    n1 = lor["n_scatter_p1"].to_numpy(int)
    n2 = lor["n_scatter_p2"].to_numpy(int)
    N_no_scatter = int((n1 == 0).sum() + (n2 == 0).sum())
    N_scattered  = int((n1 > 0).sum()  + (n2 > 0).sum())
    N_absorbed = int(N_emitted - N_exited)
    stats_by_R[R] = dict(
        emitted=N_emitted,
        exited=N_exited,
        absorbed=N_absorbed,
        no_scatter=N_no_scatter,
        scattered=N_scattered,
    )
# Make a DataFrame to see what the data looks like ,a bit difficult in the printout above.
def cell(n, denom):
    return f"{fmt_int(n)} ({fmt_pct(pct(n, denom))})"
rows = []
for name in cat_names:
    row = {"Category": name}
    for R in Rs:
        s = stats_by_R[R]
        denom = s["emitted"]
        if name == "Not scattered":
            row[f"R = {R:.0f} cm"] = cell(s["no_scatter"], denom)
        elif name == "Scattered":
            row[f"R = {R:.0f} cm"] = cell(s["scattered"], denom)
        elif name == "Absorbed":
            row[f"R = {R:.0f} cm"] = cell(s["absorbed"], denom)
        elif name == "Exited":
            row[f"R = {R:.0f} cm"] = cell(s["exited"], denom)
        elif name == "Emitted":
            row[f"R = {R:.0f} cm"] = f"{fmt_int(s['emitted'])} (100.0%)"

    rows.append(row)

df = pd.DataFrame(rows)
styled = (
    df.style
    .hide(axis="index")
    .set_properties(**{"text-align": "left"})
    .set_table_styles([
        {"selector": "th", "props": [("text-align", "left")]},
        {"selector": "td", "props": [("text-align", "left")]},
    ])
)
display(styled)


Category,R = 5 cm,R = 10 cm,R = 15 cm
Not scattered,"10,460,209 (52.3%)","5,945,323 (29.7%)","3,614,886 (18.1%)"
Scattered,"9,402,391 (47.0%)","12,834,479 (64.2%)","13,034,898 (65.2%)"
Absorbed,"137,400 (0.7%)","1,220,198 (6.1%)","3,350,216 (16.8%)"
Exited,"19,862,600 (99.3%)","18,779,802 (93.9%)","16,649,784 (83.2%)"
Emitted,"20,000,000 (100.0%)","20,000,000 (100.0%)","20,000,000 (100.0%)"
